In [2]:
import sys
print(sys.executable)
print("\nInstalled packages:")
import subprocess
result = subprocess.run([sys.executable, '-m', 'pip', 'list'], capture_output=True, text=True)
!{sys.executable} -m pip install boto3
print(result.stdout)

/Users/mac/Desktop/MyDesktop/Work/Scripts/.venv/bin/python

Installed packages:
  Using cached boto3-1.42.33-py3-none-any.whl.metadata (6.8 kB)
  Using cached botocore-1.42.33-py3-none-any.whl.metadata (5.9 kB)
  Using cached jmespath-1.1.0-py3-none-any.whl.metadata (7.6 kB)
  Using cached s3transfer-0.16.0-py3-none-any.whl.metadata (1.7 kB)
  Using cached urllib3-2.6.3-py3-none-any.whl.metadata (6.9 kB)
Using cached boto3-1.42.33-py3-none-any.whl (140 kB)
Using cached botocore-1.42.33-py3-none-any.whl (14.6 MB)
Using cached jmespath-1.1.0-py3-none-any.whl (20 kB)
Using cached s3transfer-0.16.0-py3-none-any.whl (86 kB)
Using cached urllib3-2.6.3-py3-none-any.whl (131 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [boto3]32m3/5 [s3transfer]
Package                 Version
----------------------- -----------
appnope                 0.1.4
asttokens               3.0.1
comm                    0.2.3
debugpy                 1.8.19
decorator               5.2.1
executing               2.

In [3]:
import boto3
from decimal import Decimal
import json

# Initialize DynamoDB client
dynamodb = boto3.resource('dynamodb', region_name='us-east-1')  # Change region as needed
table = dynamodb.Table('TradePricelists')

In [4]:
def scan_pricelists(supplier_id='SEI', batch_size=100):
    """
    Scan DynamoDB table for unique Pricelists for a given SupplierID.
    
    Args:
        supplier_id: The SupplierID to filter (default: 'SEI')
        batch_size: Number of items to process per batch (default: 100)
    
    Returns:
        set: Unique pricelist values
    """
    unique_pricelists = set()
    last_evaluated_key = None
    total_scanned = 0
    total_processed = 0
    
    print(f"Starting scan for SupplierID: {supplier_id}")
    print(f"Batch size: {batch_size}")
    print("-" * 50)
    
    while True:
        # Prepare scan parameters
        scan_params = {
            'FilterExpression': 'SupplierID = :sid AND attribute_not_exists(Deleted) OR (attribute_exists(Deleted) AND Deleted <> :true_val)',
            'ExpressionAttributeValues': {
                ':sid': supplier_id,
                ':true_val': True
            },
            'ProjectionExpression': 'Pricelist',
            'Limit': batch_size
        }
        
        # Add pagination token if exists
        if last_evaluated_key:
            scan_params['ExclusiveStartKey'] = last_evaluated_key
        
        # Execute scan
        response = table.scan(**scan_params)
        
        # Process items in current batch
        items = response.get('Items', [])
        batch_count = 0
        
        for item in items:
            # Check if Pricelist field exists and add to set
            if 'Pricelist' in item:
                pricelist_value = item['Pricelist']
                # Convert Decimal to string if needed
                if isinstance(pricelist_value, Decimal):
                    pricelist_value = str(pricelist_value)
                unique_pricelists.add(pricelist_value)
                batch_count += 1
        
        total_scanned += response.get('ScannedCount', 0)
        total_processed += len(items)
        
        print(f"Batch: Scanned {response.get('ScannedCount', 0)} items, "
              f"Processed {len(items)} items, "
              f"Found {batch_count} new pricelists")
        print(f"Total unique pricelists so far: {len(unique_pricelists)}")
        
        # Check if there are more items to scan
        last_evaluated_key = response.get('LastEvaluatedKey')
        if not last_evaluated_key:
            break
        
        print(f"Continuing to next batch...")
        print("-" * 50)
    
    print(f"\nScan complete!")
    print(f"Total items scanned: {total_scanned}")
    print(f"Total items processed: {total_processed}")
    print(f"Total unique pricelists found: {len(unique_pricelists)}")
    
    return unique_pricelists

def save_pricelists_to_file(pricelists, filename='unique_pricelists.txt'):
    """
    Save unique pricelists to a text file.
    
    Args:
        pricelists: Set of unique pricelist values
        filename: Output filename (default: 'unique_pricelists.txt')
    """
    # Sort pricelists for better readability
    sorted_pricelists = sorted(pricelists)
    
    with open(filename, 'w') as f:
        f.write(f"Unique Pricelists for SupplierID: SEI\n")
        f.write(f"Total Count: {len(sorted_pricelists)}\n")
        f.write("=" * 50 + "\n\n")
        
        for pricelist in sorted_pricelists:
            f.write(f"{pricelist}\n")
    
    print(f"\nPricelists saved to: {filename}")

In [5]:
def main():
    """Main execution function"""
    try:
        # Scan for unique pricelists
        unique_pricelists = scan_pricelists(supplier_id='SEI', batch_size=100)
        
        # Save to file
        save_pricelists_to_file(unique_pricelists)
        
        # Display sample results
        print("\nSample Pricelists (first 10):")
        for pricelist in sorted(list(unique_pricelists)[:10]):
            print(f"  - {pricelist}")
        
    except Exception as e:
        print(f"Error occurred: {str(e)}")
        raise

if __name__ == "__main__":
    main()

Starting scan for SupplierID: SEI
Batch size: 100
--------------------------------------------------
Batch: Scanned 100 items, Processed 100 items, Found 0 new pricelists
Total unique pricelists so far: 0
Continuing to next batch...
--------------------------------------------------
Batch: Scanned 100 items, Processed 100 items, Found 0 new pricelists
Total unique pricelists so far: 0
Continuing to next batch...
--------------------------------------------------
Batch: Scanned 100 items, Processed 100 items, Found 0 new pricelists
Total unique pricelists so far: 0
Continuing to next batch...
--------------------------------------------------
Batch: Scanned 100 items, Processed 100 items, Found 0 new pricelists
Total unique pricelists so far: 0
Continuing to next batch...
--------------------------------------------------
Batch: Scanned 100 items, Processed 100 items, Found 0 new pricelists
Total unique pricelists so far: 0
Continuing to next batch...
-----------------------------------

KeyboardInterrupt: 